In [12]:
import kagglehub
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import shutil

In [ ]:
# 读取数据
dirty_path = "dirty_cafe_sales.csv"
clean_path = "clean_cafe_sales.csv"

shutil.copyfile(dirty_path, clean_path)

dirty_df = pd.read_csv(dirty_path)
clean_df = pd.read_csv(clean_path)

dirty_df.head()



,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [16]:
clean_df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [31]:
duplicate_ids = clean_df[clean_df['Transaction ID'].duplicated(keep=False)]

if not duplicate_ids.empty:
    print("⚠️ 检测到重复的 Transaction ID：")
    print(duplicate_ids[['Transaction ID', 'Item', 'Total Spent']])
    
    # 删除重复的 Transaction ID，只保留第一次出现的记录
    data = clean_df.drop_duplicates(subset=['Transaction ID'], keep='first')
    print(f"✅ 已删除重复 Transaction ID，共删除 {duplicate_ids['Transaction ID'].nunique()} 条。")
else:
    print("✅ 没有发现重复的 Transaction ID。")

✅ 没有发现重复的 Transaction ID。


In [27]:
def check_data_quality(df):
    """
    检查 DataFrame 各列中 'UNKNOWN'、'ERROR' 和 NaN 的数量。
    """
    keywords = ["UNKNOWN", "ERROR"]
    result = {}

    for col in df.columns:
        # 转换为字符串以便匹配
        col_str = df[col].astype(str).str.upper()
        result[col] = {
            "UNKNOWN": col_str.str.contains("UNKNOWN").sum(),
            "ERROR": col_str.str.contains("ERROR").sum(),
            "NaN": df[col].isna().sum()
        }

    # 打印结果
    print("📊 数据质量检查报告")
    print("-" * 50)
    for col, counts in result.items():
        print(f"{col:20s} | UNKNOWN: {counts['UNKNOWN']:3d} | ERROR: {counts['ERROR']:3d} | NaN: {counts['NaN']:3d}")
    print("-" * 50)

    



In [28]:
check_data_quality(dirty_df)

📊 数据质量检查报告
--------------------------------------------------
Transaction ID       | UNKNOWN:   0 | ERROR:   0 | NaN:   0
Item                 | UNKNOWN: 344 | ERROR: 292 | NaN: 333
Quantity             | UNKNOWN: 171 | ERROR: 170 | NaN: 138
Price Per Unit       | UNKNOWN: 164 | ERROR: 190 | NaN: 179
Total Spent          | UNKNOWN: 165 | ERROR: 164 | NaN: 173
Payment Method       | UNKNOWN: 293 | ERROR: 306 | NaN: 2579
Location             | UNKNOWN: 338 | ERROR: 358 | NaN: 3265
Transaction Date     | UNKNOWN: 159 | ERROR: 142 | NaN: 159
--------------------------------------------------


In [29]:
# 将 Total Spent 中的 ERROR 替换为 NaN
clean_df['Total Spent'] = pd.to_numeric(clean_df['Total Spent'], errors='coerce')
clean_df['Quantity'] = pd.to_numeric(clean_df['Quantity'], errors='coerce')
clean_df['Price Per Unit'] = pd.to_numeric(clean_df['Price Per Unit'], errors='coerce')



# 修复 Total Spent = NaN 的值
clean_df['Total Spent'] = clean_df['Total Spent'].fillna(clean_df['Quantity'] * clean_df['Price Per Unit'])




In [30]:
check_data_quality(clean_df)

📊 数据质量检查报告
--------------------------------------------------
Transaction ID       | UNKNOWN:   0 | ERROR:   0 | NaN:   0
Item                 | UNKNOWN: 344 | ERROR: 292 | NaN: 333
Quantity             | UNKNOWN:   0 | ERROR:   0 | NaN: 479
Price Per Unit       | UNKNOWN:   0 | ERROR:   0 | NaN: 533
Total Spent          | UNKNOWN:   0 | ERROR:   0 | NaN:  40
Payment Method       | UNKNOWN: 293 | ERROR: 306 | NaN: 2579
Location             | UNKNOWN: 338 | ERROR: 358 | NaN: 3265
Transaction Date     | UNKNOWN: 159 | ERROR: 142 | NaN: 159
--------------------------------------------------


In [38]:
import pandas as pd

# 假设 data 是你的 DataFrame
# 确保列是正确的类型
clean_df['Price Per Unit'] = pd.to_numeric(clean_df['Price Per Unit'], errors='coerce')

# 1️⃣ 检查每个商品的唯一单价数量
price_variation = clean_df.groupby('Item')['Price Per Unit'].nunique().reset_index()
price_variation.columns = ['Item', 'Unique_Price_Count']

# 2️⃣ 找出存在多个单价的商品
price_changed_items = price_variation[price_variation['Unique_Price_Count'] > 1]

print("🧾 每个商品的单价变化情况：")
print(price_variation)

print("\n⚠️ 以下商品存在单价不一致：")
print(price_changed_items)

item_prices = clean_df.groupby('Item')['Price Per Unit'].unique().reset_index()
print(item_prices)


🧾 每个商品的单价变化情况：
       Item  Unique_Price_Count
0      Cake                   1
1    Coffee                   1
2    Cookie                   1
3     ERROR                   6
4     Juice                   1
5     Salad                   1
6  Sandwich                   1
7  Smoothie                   1
8       Tea                   1
9   UNKNOWN                   6

⚠️ 以下商品存在单价不一致：
      Item  Unique_Price_Count
3    ERROR                   6
9  UNKNOWN                   6
       Item                       Price Per Unit
0      Cake                           [3.0, nan]
1    Coffee                           [2.0, nan]
2    Cookie                           [1.0, nan]
3     ERROR  [1.5, 3.0, 5.0, nan, 4.0, 2.0, 1.0]
4     Juice                           [3.0, nan]
5     Salad                           [5.0, nan]
6  Sandwich                           [4.0, nan]
7  Smoothie                           [4.0, nan]
8       Tea                           [1.5, nan]
9   UNKNOWN  [3.0, 1.0, 5.0, 4.0,

In [45]:
valid_items = clean_df[~clean_df['Item'].isin(['UNKNOWN', 'ERROR'])]  # 排除异常Item
item_price_map = valid_items.groupby('Item')['Price Per Unit'].first().to_dict()


# 查看字典内容
print("Item 对应价格字典：")
print(item_price_map)



Item 对应价格字典：
{'Cake': 3.0, 'Coffee': 2.0, 'Cookie': 1.0, 'Juice': 3.0, 'Salad': 5.0, 'Sandwich': 4.0, 'Smoothie': 4.0, 'Tea': 1.5}


In [46]:
def fill_price_per_unit(df, item_col='Item', price_col='Price Per Unit', skip_items=None, method='first'):
    """
    通用函数：按 Item 填充 Price Per Unit 的 NaN 值，跳过指定的异常 Item。

    参数：
    df : pd.DataFrame - 数据集
    item_col : str - 商品列名
    price_col : str - 单价列名
    skip_items : list - 不进行填充的异常商品列表，默认 ['UNKNOWN','ERROR']
    method : str - 参考价格获取方法，'first' 或 'mean'

    返回：
    df : pd.DataFrame - 填充后的数据集
    """
    if skip_items is None:
        skip_items = ['UNKNOWN', 'ERROR']

    # 确保价格列为数值型
    df[price_col] = pd.to_numeric(df[price_col], errors='coerce')

    # 生成参考价格，只使用正常商品
    valid_items = df[~df[item_col].isin(skip_items)]
    if method == 'first':
        item_price_map = valid_items.groupby(item_col)[price_col].first().to_dict()
    elif method == 'mean':
        item_price_map = valid_items.groupby(item_col)[price_col].mean().to_dict()
    else:
        raise ValueError("method 参数必须是 'first' 或 'mean'")

    # 定义填充函数
    def fill_price(row):
        if pd.isna(row[price_col]) and row[item_col] not in skip_items:
            return item_price_map.get(row[item_col], np.nan)
        else:
            return row[price_col]

    # 执行填充
    df[price_col] = df.apply(fill_price, axis=1)
    return df

In [48]:
# 假设 data 是你的 DataFrame
fill_price_per_unit(clean_df, item_col='Item', price_col='Price Per Unit', skip_items=['UNKNOWN','ERROR'], method='first')

# 查看结果
print("填充后 Price Per Unit 中的缺失值数量:", clean_df['Price Per Unit'].isna().sum())
print(clean_df.head(10))


填充后 Price Per Unit 中的缺失值数量: 54
  Transaction ID      Item  Quantity  Price Per Unit  Total Spent  \
0    TXN_1961373    Coffee       2.0             2.0          4.0   
1    TXN_4977031      Cake       4.0             3.0         12.0   
2    TXN_4271903    Cookie       4.0             1.0          4.0   
3    TXN_7034554     Salad       2.0             5.0         10.0   
4    TXN_3160411    Coffee       2.0             2.0          4.0   
5    TXN_2602893  Smoothie       5.0             4.0         20.0   
6    TXN_4433211   UNKNOWN       3.0             3.0          9.0   
7    TXN_6699534  Sandwich       4.0             4.0         16.0   
8    TXN_4717867       NaN       5.0             3.0         15.0   
9    TXN_2064365  Sandwich       5.0             4.0         20.0   

   Payment Method  Location Transaction Date  
0     Credit Card  Takeaway       2023-09-08  
1            Cash  In-store       2023-05-16  
2     Credit Card  In-store       2023-07-19  
3         UNKNOWN   U